In [340]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [341]:
## LOAD THIS DATA (sitll tanh normalized)
df = pd.read_csv("final_video_data_normalized.csv")

In [342]:
## Drop so keras doesn't try to use the title as a feature
df = df.drop(columns=['title'])


In [343]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [344]:
train.head()

,view_count,percent_capital,title_length,contains_question_mark,contains_exclamation_mark,VADER,has_full_cap_word,full_cap_count,trending
3382,309045,0.142857,0.494442,False,False,0.507467,False,0.496823,True
5646,21,0.708861,0.506825,False,False,0.499709,True,0.531282,False
20156,34,0.000000,0.491286,False,False,0.499709,False,0.496823,False
17,410704,0.119403,0.503912,False,False,0.503264,False,0.496823,True
3579,53233,0.156250,0.495413,False,False,0.499709,True,0.499287,False


In [345]:
test.head()

,view_count,percent_capital,title_length,contains_question_mark,contains_exclamation_mark,VADER,has_full_cap_word,full_cap_count,trending
15,644,0.666667,0.488374,False,False,0.499709,True,0.499287,False
9049,10350981,0.147059,0.504155,False,False,0.492503,False,0.496823,True
13128,2920,0.142857,0.496142,False,False,0.499709,False,0.496823,False
11320,145849,0.571429,0.504640,False,True,0.499709,True,0.526370,False
16909,3512189,0.133333,0.498570,False,False,0.499709,False,0.496823,True


In [346]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('trending')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        #ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [347]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

In [348]:
feature_columns = []
for header in ['percent_capital', 'title_length', 'contains_exclamation_mark', 'contains_question_mark', 'has_full_cap_word']:
    feature_columns.append(tf.feature_column.numeric_column(header))
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [349]:
model = tf.keras.Sequential([
  feature_layer,
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


In [350]:
opt = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer = opt,loss='binary_crossentropy', metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [351]:
model.fit(train_ds, epochs = 10, callbacks=[tensorboard_callback])

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
525/525 [==============================] - 2s 3ms/step - loss: 0.6637 - accuracy: 0.6242
Epoch 2/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6411 - accuracy: 0.6536
Epoch 3/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6261 - accuracy: 0.6620
Epoch 4/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6228 - accuracy: 0.6637
Epoch 5/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6206 - accuracy: 0.6643
Epoch 6/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6191 - accuracy: 0.6649
Epoch 7/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6184 - accuracy: 0.6664
Epoch 8/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6180 - accuracy: 0.6655
Epoch 9/10
525/525 [==============================] - 1s 1ms/step - loss: 0.6181 - accurac

In [352]:
model.evaluate(test_ds)

Consider rewriting this model with the Functional API.
132/132 [==============================] - 0s 1ms/step - loss: 0.6031 - accuracy: 0.6780


[0.6031421422958374, 0.6780348420143127]

In [353]:
ynew = model.predict(test_ds)

Consider rewriting this model with the Functional API.


In [354]:
ynew

array([[0.3701902 ],
       [0.78803766],
       [0.7869374 ],
       ...,
       [0.3825125 ],
       [0.3831899 ],
       [0.33053738]], dtype=float32)

In [355]:
ynew = np.where(ynew<.5,0,1)

In [356]:
ynew

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [359]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
true_test = np.concatenate([y for x, y in test_ds], axis=0)
tn, fp, fn, tp = confusion_matrix(true_test, ynew).ravel()
(tn, fp, fn, tp)

(1699, 397, 953, 1144)

In [358]:
%tensorboard --logdir logs/fit

In [362]:
print(classification_report(true_test, ynew))

              precision    recall  f1-score   support

       False       0.64      0.81      0.72      2096
        True       0.74      0.55      0.63      2097

    accuracy                           0.68      4193
   macro avg       0.69      0.68      0.67      4193
weighted avg       0.69      0.68      0.67      4193

